# Unstructured and Structured RAG with Intelligent Query Routing

This notebook demonstrates an advanced RAG pattern that intelligently routes queries between **structured** and **unstructured** knowledge bases using **Strands Agents** 

## Overview

The system uses custom retrieval tools that connect to Amazon Bedrock Knowledge Bases:
1. **Query Analysis**: Agent analyzes incoming query to determine type
2. **Tool Selection**: Routes to unstructured (document) or structured (SQL) assistant
3. **Retrieval**: Appropriate knowledge base retrieves relevant information
4. **Response Generation**: Agent synthesizes and presents the results

This approach enables handling both qualitative questions (business strategy, policies) and quantitative queries (financial metrics, data analysis) within a single conversational interface.

## Prerequisites and Setup

Before running this notebook, ensure you have completed:
1. **`0-prerequisites-structured-kb.ipynb`** - Sets up Redshift-based structured Amazon Bedrock Knowledge Base
2. **`1-prerequisites-unstructured-kb.ipynb`** - Sets up document-based unstructured Amazon Bedrock Knowledge Base


Let's start by importing the required libraries:


In [2]:
import os
import boto3
from strands import Agent, tool

In [3]:
# Set up AWS region and  Amazon Bedrock Agent Runtime client for knowledge base interactions
region = boto3.Session().region_name 
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name=region)

### Load Knowledge Base Configuration

Load the knowledge base IDs from the prerequisite notebooks:


In [4]:
UNSTRUCTURED_KB_ID = "FJ6OIBKNMP"  
STRUCTURED_KB_ID = "G5F1E2PFCI"  
# # Retrieve stored knowledge base IDs from prerequisite notebooks
# %store -r unstructured_kb_id
# %store -r structured_kb_id 
# %store -r kb_region
# %store -r structured_kb_region

# # Use the stored values
# UNSTRUCTURED_KB_ID = unstructured_kb_id  # From 1-prerequisites-unstructured-kb.ipynb
# STRUCTURED_KB_ID = structured_kb_id      # From 0-prerequisites-structured-kb.ipynb


# print("="*60)
# print(f"Unstructured KB ID: {UNSTRUCTURED_KB_ID}")
# print(f"Structured KB ID: {STRUCTURED_KB_ID}")
# print(f"Unstructured KB Region: {kb_region}")
# print(f"Structured KB Region: {structured_kb_region}")


## Custom Retrieval Tools

We will create two specialized tools using the Strands Agents `@tool` decorator. Each tool handles different types of queries by connecting to the appropriate knowledge base.

### Unstructured Data Assistant Tool

This tool handles document-based queries by retrieving information from the unstructured knowledge base (PDF documents, reports, policies):


In [5]:
@tool
def unstructured_data_assistant(query: str) -> str:
    """
    Handle document-based, narrative, and conceptual queries using the unstructured knowledge base.
    
    Args:
        query: A question about business strategies, policies, company information, 
               or requiring document comprehension and qualitative analysis
    
    Returns:
        Raw retrieve response from the unstructured knowledge base
    """
    try:
        retrieve_response = bedrock_agent_runtime.retrieve(
            knowledgeBaseId=UNSTRUCTURED_KB_ID,
            retrievalQuery={'text': query},
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': 10,
                }
            }
        )
        
        return retrieve_response
        
    except Exception as e:
        return f"Error in unstructured data assistant: {str(e)}"


### Structured Data Assistant Tool

This tool handles data analysis queries by retrieving information from the structured knowledge base (SQL/Redshift database):


In [6]:
@tool
def structured_data_assistant(query: str) -> str:
    """
    Handle data analysis, metrics, and quantitative queries using the structured knowledge base.
    
    Args:
        query: A question requiring calculations, aggregations, statistical analysis,
               or database operations on structured data
    
    Returns:
        Raw retrieve response from the structured knowledge base
    """
    try:
        retrieve_response = bedrock_agent_runtime.retrieve(
            knowledgeBaseId=STRUCTURED_KB_ID,
            retrievalQuery={'text': query},
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': 10,
                }
            }
        )
        
        return retrieve_response
        
    except Exception as e:
        return f"Error in structured data assistant: {str(e)}"


## Intelligent Agent with Query Routing

Now we'll create the orchestrator agent with our custom tools and system prompt that intelligently routes queries to the appropriate tool based on the query type and content.


In [7]:
# Create the orchestrator agent with both tools
orchestrator = Agent(
    system_prompt="""You are an intelligent assistant that routes queries to the appropriate knowledge base. Choose the appropriate tool based on the query type. 
    The tools return raw data that you should analyze and present in a clear, helpful format.""",
    tools=[
        unstructured_data_assistant,
        structured_data_assistant
    ]
)

Let's test our agent with different types of queries to observe how it routes them to the appropriate knowledge bases.

### Example 1: Unstructured Query - Business Strategy

This query asks about qualitative, document-based information and should be routed to the unstructured knowledge base:


In [8]:
# EXAMPLE 1: Business Strategy Query (should use unstructured_data_assistant)
print("=== EXAMPLE 1: BUSINESS STRATEGY QUERY ===")
print("Query: What is Octank Financial's business strategy?")
print()

response = orchestrator("What is Octank Financial's business strategy?")
print(response)

=== EXAMPLE 1: BUSINESS STRATEGY QUERY ===
Query: What is Octank Financial's business strategy?

I'll help you find information about Octank Financial's business strategy by querying our knowledge base. This is a question about company information and business strategies, so I'll use the appropriate tool for that.
Tool #1: unstructured_data_assistant
# Octank Financial's Business Strategy

Based on the information retrieved from Octank Financial's documentation, here's a comprehensive overview of their business strategy:

## Core Business Focus
Octank Financial is a leading financial services company that provides a wide range of services including:
- Investment banking
- Wealth management
- Asset management
- Corporate finance
- Private equity

## Strategic Pillars

### 1. Client-Centric Approach
- Building long-term relationships with clients based on trust, transparency, and mutual respect
- Delivering exceptional service tailored to clients' unique needs and goals
- Working closely

### Example 2: Structured Query - Financial Metrics

This query asks for quantitative data analysis and should be routed to the structured knowledge base:


In [9]:
# EXAMPLE 2: Financial Data Query (should use structured_data_assistant)
print("=== EXAMPLE 2: FINANCIAL DATA QUERY ===")
print("Query: What is the total spending by all customers?")
print()

response = orchestrator("What is the total spending by all customers?")
print(response)


=== EXAMPLE 2: FINANCIAL DATA QUERY ===
Query: What is the total spending by all customers?

I'll help you find information about the total spending by all customers of Octank Financial. This question requires analysis of numerical data, so I'll use the structured data assistant to retrieve this information.
Tool #2: structured_data_assistant
# Total Customer Spending

Based on the query results from Octank Financial's database, the total spending by all customers is:

**$5,078,473.69** (Five million, seventy-eight thousand, four hundred seventy-three dollars and sixty-nine cents)

This figure represents the sum of all order totals in the company's transaction records.# Total Customer Spending

Based on the query results from Octank Financial's database, the total spending by all customers is:

**$5,078,473.69** (Five million, seventy-eight thousand, four hundred seventy-three dollars and sixty-nine cents)

This figure represents the sum of all order totals in the company's transaction

## Agent Thinking Inspection

One of the powerful features of Strands Agents is the ability to inspect the agent's internal reasoning process. Let's examine how the agent made its decisions:

### Message History Analysis


In [10]:
orchestrator.messages

[{'role': 'user',
  'content': [{'text': "What is Octank Financial's business strategy?"}]},
 {'role': 'assistant',
  'content': [{'text': "I'll help you find information about Octank Financial's business strategy by querying our knowledge base. This is a question about company information and business strategies, so I'll use the appropriate tool for that."},
   {'toolUse': {'toolUseId': 'tooluse_SsBe3pmZT5qzUkJEyqtNXw',
     'name': 'unstructured_data_assistant',
     'input': {'query': "What is Octank Financial's business strategy?"}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_SsBe3pmZT5qzUkJEyqtNXw',
     'status': 'success',
     'content': [{'text': '{\'ResponseMetadata\': {\'RequestId\': \'38db5fab-cb7d-48c7-90a2-3a169b087add\', \'HTTPStatusCode\': 200, \'HTTPHeaders\': {\'date\': \'Mon, 23 Jun 2025 02:01:30 GMT\', \'content-type\': \'application/json\', \'content-length\': \'19778\', \'connection\': \'keep-alive\', \'x-amzn-requestid\': \'38db5fab-

### Tool Usage Analysis

Let's analyze which tools were used for each query type and examine the decision-making process:


In [11]:
# Analyze tool usage across conversation history
def analyze_tool_usage(messages):
    tool_usage = []
    
    for i, message in enumerate(messages):
        if message['role'] == 'assistant':
            for content in message['content']:
                if 'toolUse' in content:
                    tool_name = content['toolUse']['name']
                    tool_input = content['toolUse']['input']
                    tool_usage.append({
                        'message_index': i,
                        'tool_name': tool_name,
                        'query': tool_input.get('query', ''),
                        'tool_id': content['toolUse']['toolUseId']
                    })
    
    return tool_usage

# Analyze the conversation
tool_usage = analyze_tool_usage(orchestrator.messages)

print("=== TOOL USAGE ANALYSIS ===")
for usage in tool_usage:
    print(f"Tool: {usage['tool_name']}")
    print(f"Query: {usage['query']}")
    print(f"Tool ID: {usage['tool_id']}")
    print("-" * 50)


=== TOOL USAGE ANALYSIS ===
Tool: unstructured_data_assistant
Query: What is Octank Financial's business strategy?
Tool ID: tooluse_SsBe3pmZT5qzUkJEyqtNXw
--------------------------------------------------
Tool: structured_data_assistant
Query: What is the total spending by all customers?
Tool ID: tooluse_ZsuuH7t7RAeZBk-AZ_Uoqw
--------------------------------------------------


### Detailed Message Flow Inspection

Let's examine the complete message flow to understand the agent's reasoning process:


In [12]:
# Inspect the complete conversation flow
def inspect_message_flow(messages):
    print("=== DETAILED MESSAGE FLOW ===")
    
    for i, message in enumerate(messages):
        print(f"\n--- Message {i+1} ---")
        print(f"Role: {message['role']}")
        
        for j, content in enumerate(message['content']):
            print(f"  Content {j+1}:")
            
            if 'text' in content:
                text = content['text']
                # Truncate long text for readability
                if len(text) > 200:
                    text = text[:200] + "..."
                print(f"    Text: {text}")
            
            elif 'toolUse' in content:
                tool_use = content['toolUse']
                print(f"    Tool Use: {tool_use['name']}")
                print(f"    Input: {tool_use['input']}")
                print(f"    ID: {tool_use['toolUseId']}")
            
            elif 'toolResult' in content:
                tool_result = content['toolResult']
                print(f"    Tool Result: {tool_result['status']}")
                print(f"    ID: {tool_result['toolUseId']}")
                # Don't print full content as it's very long
                print(f"    Content: [Raw KB Response - {len(str(tool_result['content']))} chars]")

# Run the inspection
inspect_message_flow(orchestrator.messages)


=== DETAILED MESSAGE FLOW ===

--- Message 1 ---
Role: user
  Content 1:
    Text: What is Octank Financial's business strategy?

--- Message 2 ---
Role: assistant
  Content 1:
    Text: I'll help you find information about Octank Financial's business strategy by querying our knowledge base. This is a question about company information and business strategies, so I'll use the appropri...
  Content 2:
    Tool Use: unstructured_data_assistant
    Input: {'query': "What is Octank Financial's business strategy?"}
    ID: tooluse_SsBe3pmZT5qzUkJEyqtNXw

--- Message 3 ---
Role: user
  Content 1:
    Tool Result: success
    ID: tooluse_SsBe3pmZT5qzUkJEyqtNXw
    Content: [Raw KB Response - 20776 chars]

--- Message 4 ---
Role: assistant
  Content 1:
    Text: # Octank Financial's Business Strategy

Based on the information retrieved from Octank Financial's documentation, here's a comprehensive overview of their business strategy:

## Core Business Focus
Oc...

--- Message 5 ---
Role: user
 

## Advanced Query Testing

Let's test the system with more complex queries to validate the routing logic:


In [ ]:
# Test additional queries to validate routing logic
test_queries = [
    {
        "query": "What are Octank Financial's core values?",
        "expected_tool": "unstructured_data_assistant",
        "type": "Qualitative/Policy"
    },
    {
        "query": "How many orders were placed by customers in total?",
        "expected_tool": "structured_data_assistant", 
        "type": "Quantitative/Count"
    },
    {
        "query": "What is the company's approach to corporate social responsibility?",
        "expected_tool": "unstructured_data_assistant",
        "type": "Qualitative/Strategy"
    },
    {
        "query": "What is the average order value across all customers?",
        "expected_tool": "structured_data_assistant",
        "type": "Quantitative/Analysis"
    }
]

print("=== ADVANCED QUERY ROUTING VALIDATION ===\n")

for i, test_case in enumerate(test_queries, 1):
    print(f"TEST {i}: {test_case['type']}")
    print(f"Query: {test_case['query']}")
    print(f"Expected Tool: {test_case['expected_tool']}")
    
    # Get response and check routing
    response = orchestrator(test_case['query'])
    
    # Check which tool was actually used
    recent_messages = orchestrator.messages[-2:]  # Last user message and assistant response
    used_tool = None
    
    for message in recent_messages:
        if message['role'] == 'assistant':
            for content in message['content']:
                if 'toolUse' in content:
                    used_tool = content['toolUse']['name']
                    break
    
    routing_correct = used_tool == test_case['expected_tool']
    status = "✅ CORRECT" if routing_correct else "❌ INCORRECT"
    
    print(f"Actual Tool Used: {used_tool}")
    print(f"Routing Status: {status}")
    print(f"Response Length: {len(response)} characters")
    print("-" * 80)
    print()


## System Performance Analysis

Let's analyze the overall performance and routing accuracy of our dual knowledge base system:


In [13]:
# Generate performance summary
def generate_performance_summary():
    total_interactions = len([msg for msg in orchestrator.messages if msg['role'] == 'user'])
    tool_usage_counts = {}
    
    # Count tool usage
    for message in orchestrator.messages:
        if message['role'] == 'assistant':
            for content in message['content']:
                if 'toolUse' in content:
                    tool_name = content['toolUse']['name']
                    tool_usage_counts[tool_name] = tool_usage_counts.get(tool_name, 0) + 1
    
    print("=== DUAL KNOWLEDGE BASE RAG SYSTEM SUMMARY ===")
    print(f"📊 Total User Queries Processed: {total_interactions}")
    print(f"🔧 Total Tool Invocations: {sum(tool_usage_counts.values())}")
    print()
    
    print("🎯 Tool Usage Breakdown:")
    for tool_name, count in tool_usage_counts.items():
        percentage = (count / sum(tool_usage_counts.values())) * 100
        print(f"  • {tool_name}: {count} invocations ({percentage:.1f}%)")
    
    print()
    print("📈 System Capabilities Demonstrated:")
    print("  ✅ Intelligent query routing between structured and unstructured data")
    print("  ✅ Natural language interface for both SQL and document queries")
    print("  ✅ Comprehensive response synthesis from multiple knowledge sources")
    print("  ✅ Transparent agent reasoning and tool selection process")
    
    return {
        'total_queries': total_interactions,
        'tool_usage': tool_usage_counts,
        'conversation_length': len(orchestrator.messages)
    }

summary = generate_performance_summary()


=== DUAL KNOWLEDGE BASE RAG SYSTEM SUMMARY ===
📊 Total User Queries Processed: 4
🔧 Total Tool Invocations: 2

🎯 Tool Usage Breakdown:
  • unstructured_data_assistant: 1 invocations (50.0%)
  • structured_data_assistant: 1 invocations (50.0%)

📈 System Capabilities Demonstrated:
  ✅ Intelligent query routing between structured and unstructured data
  ✅ Natural language interface for both SQL and document queries
  ✅ Comprehensive response synthesis from multiple knowledge sources
  ✅ Transparent agent reasoning and tool selection process


## Key Insights and Technical Architecture

### 🏗️ **Technical Implementation Highlights**

This dual knowledge base RAG system demonstrates several advanced concepts:

1. **Multi-Modal Knowledge Integration**: Successfully combines structured (SQL/Redshift) and unstructured (PDF/documents) data sources
2. **Intelligent Query Classification**: Uses LLM reasoning to determine query intent and route appropriately
3. **Tool-Based Architecture**: Leverages Strands framework's `@tool` decorator for modular, reusable components
4. **Raw Data Processing**: Agent synthesizes raw knowledge base responses into coherent, user-friendly answers

### 🔍 **Agent Decision-Making Process**

The agent follows this reasoning pattern:
1. **Query Analysis**: Examines the semantic content and intent
2. **Classification**: Determines if the query requires qualitative (document) or quantitative (data) information  
3. **Tool Selection**: Routes to the appropriate knowledge base assistant
4. **Response Synthesis**: Processes raw retrieval results and presents insights clearly

### 🎯 **Use Cases and Applications**

This architecture is ideal for scenarios requiring:
- **Enterprise Knowledge Management**: Combining policy documents with operational data
- **Financial Analysis**: Mixing quantitative metrics with qualitative research
- **Customer Support**: Accessing both procedural documentation and transactional history
- **Business Intelligence**: Blending strategic documents with analytical data


## Conclusion

### ✅ **What We've Accomplished**

This notebook successfully demonstrated:

1. **Dual Knowledge Base Setup**: Integration of both structured (Redshift) and unstructured (OpenSearch) knowledge bases
2. **Intelligent Agent Orchestration**: Using Strands Agents to route queries based on semantic understanding
3. **Custom Tool Development**: Created specialized retrieval tools for different data types
4. **Transparent Decision Making**: Inspected agent reasoning and tool selection process
5. **Performance Validation**: Tested routing accuracy across multiple query types

### 🚀 **Next Steps and Extensions**

Potential enhancements to explore:
- **Hybrid Queries**: Handle queries requiring both structured and unstructured information
- **Context Preservation**: Maintain conversation context across multiple knowledge bases
- **Advanced Routing**: Implement more sophisticated query classification logic
- **Performance Optimization**: Add caching and response time optimization
- **Multi-Turn Conversations**: Support follow-up questions within the same context

### 📚 **Key Takeaways**

The Strands Agents framework provides a powerful foundation for building intelligent, multi-modal RAG systems that can seamlessly combine different types of knowledge sources while maintaining transparency in the decision-making process.

**This architecture pattern is particularly valuable for enterprise applications where users need access to both quantitative business data and qualitative documentation through a single, intelligent interface.**
